In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Checking the Equivalence of Regular Expressions

In order to check whether two regular expressions $r_1$ and $r_2$ are *equivalent*, perform the 
following steps:
- convert the regular expressions $r_1$ and $r_2$ into non-deterministic *FSMs*
  $F_1$ and $F_2$ such that $L(r_1) = L(F_1)$ and $L(r_2) = L(F_2)$,
- convert the non-deterministic *FSMs* $F_1$ and $F_2$ into deterministic *FSMs*
  $D_1$ and $D_2$ such that $L(D_1) = L(F_1)$ and $L(D_2) = L(F_2)$
- check whether both $L(D_1) \backslash L(D_2)$ and $L(D_2) \backslash L(D_1)$ are empty.

The notebook `Regexp-2-NFA.ipynb` contains the function `RegExp2NFA.toNFA` that can be used to compute a non-deterministic 
<span style="font-variant:small-caps;">Fsm</span> that accepts the language described by a given regular expression.

In [ ]:
import { RecursiveSet, Tuple } from 'recursive-set';
import { State, Char, DFA, NFA, nfa2dfa, key as fsmKey } from './01-NFA-2-DFA';
import { RegExp, RegExp2NFA } from './03-Regexp-2-NFA';

`NFA-2-DFA.ts` contains the function `nfa2dfa` that converts a non-deterministic 
*Fsm* into an equivalent deterministic *Fsm*.

In [ ]:
// Tuple erwartet ein Array-Format für Generics: Tuple<[S, S]>
type StatePair<S> = Tuple<[S, S]>;

type GenericDFA<S> = {
    Q: RecursiveSet<S>;
    Sigma: RecursiveSet<Char>;
    delta: Map<string, S>;
    q0: S;
    A: RecursiveSet<S>;
};


Given two sets `A` and `B`, the function `cartesian_product(A, B)` computes the 
<em style="color:blue">cartesian product</em> $A \times B$ which is defined as
$$ A \times B := \{ (x, y) \mid x \in A \wedge y \in B \}. $$

In [ ]:
function genKey<S>(state: S, c: Char): string {
    return `${String(state)},${c}`;
}

Given to deterministic *FSMs* `F1` and `F2`, the expression `fsm_complement(F1, F2)` computes a deterministic 
*FSM* that recognizes the language  $L(F_1)\backslash L(F_2)$.

In [ ]:
function fsm_complement<S>(
    F1: GenericDFA<S>,
    F2: GenericDFA<S>
): GenericDFA<StatePair<S>> {
    // Wir weisen Variablen einzeln zu, um Destructuring-Konflikte mit "exports" zu vermeiden
    const Q1 = F1.Q;
    const Sigma = F1.Sigma;
    const delta1 = F1.delta;
    const q1 = F1.q0;
    const A1 = F1.A;

    const Q2 = F2.Q;
    const delta2 = F2.delta;
    const q2 = F2.q0;
    const A2 = F2.A;

    // Casten des Rückgabewerts auf den erwarteten Typ
    const newStates = Q1.cartesianProduct(Q2) as RecursiveSet<StatePair<S>>;
    
    const newDelta = new Map<string, StatePair<S>>();

    for (const element of newStates) {
        // ASSERTION: Wir wissen, dass element ein Tuple ist
        const statePair = element as StatePair<S>;
        
        const p1 = statePair.get(0);
        const p2 = statePair.get(1);

        if (p1 === undefined || p2 === undefined) continue;

        for (const sigmaElement of Sigma) {
            // ASSERTION: Wir wissen, dass sigmaElement ein Char ist
            const c = sigmaElement as unknown as Char;

            const next1 = delta1.get(genKey(p1, c));
            const next2 = delta2.get(genKey(p2, c));

            if (next1 !== undefined && next2 !== undefined) {
                const nextPair = new Tuple(next1, next2);
                newDelta.set(genKey(statePair, c), nextPair);
            }
        }
    }

    const startPair = new Tuple(q1, q2);

    const diffSet = Q2.difference(A2);
    const newAccepting = A1.cartesianProduct(diffSet) as RecursiveSet<StatePair<S>>;

    return {
        Q: newStates,
        Sigma: Sigma,
        delta: newDelta,
        q0: startPair,
        A: newAccepting,
    };
}

Given a regular expression $r$ and an alphabet $\Sigma$, the function $\texttt{regexp2DFA}(r, \Sigma)$
computes a deterministic *FSM* that accepts
the language specified by $r$.

In [ ]:
function regexp2DFA(r: RegExp, Sigma: RecursiveSet<Char>): DFA {
    const converter = new RegExp2NFA(Sigma);
    const nfa = converter.toNFA(r);
    return nfa2dfa(nfa);
}

Given a deterministic *FSM* $F$ the function 
`is_empty(F)` checks whether the language accepted by $F$ is empty.
In this function, the variable `Reachable` is the set of those states that are already known to be reachable
from the start state `q0`. `NewFound` are those states that can be reached from a state in the set 
`Reachable`.  When we find no new states that are reachable, the iteration stops and we check whether
there is a state that is both reachable and acceptable because in that case the language is not empty.

In [ ]:
function is_empty<S>(F: GenericDFA<S>): boolean {
    const Sigma = F.Sigma;
    const delta = F.delta;
    const q0 = F.q0;
    const A = F.A;

    let reachable = new RecursiveSet<S>(q0);

    while (true) {
        const newFound = new RecursiveSet<S>();

        for (const q of reachable) {
            const state = q as S;

            for (const sigmaElement of Sigma) {
                const c = sigmaElement as unknown as Char;

                const target = delta.get(genKey(state, c));
                if (target !== undefined) {
                    newFound.add(target);
                }
            }
        }

        if (newFound.isSubset(reachable)) {
            break;
        }
        reachable = reachable.union(newFound);
    }

    return reachable.intersection(A).isEmpty();
}

The function `regExpEquiv` takes three arguments:
- $r_1$ and $r_2$ are regular expressions,
- $\Sigma$ is the alphabet used in these regular expressions.

The function returns `True` iff $r_1 \doteq r_2$, i.e. if $r_1$ and $r_2$ are equivalent. 

In [ ]:
function regExpEquiv(
    r1: RegExp,
    r2: RegExp,
    Sigma: RecursiveSet<Char>
): boolean {
    const F1 = regexp2DFA(r1, Sigma);
    const F2 = regexp2DFA(r2, Sigma);

    const r1MinusR2 = fsm_complement(F1, F2);
    const r2MinusR1 = fsm_complement(F2, F1);

    return is_empty(r1MinusR2) && is_empty(r2MinusR1);
}

The notebook `Test-Equivalence.ipynb` can be used to test this function.